## TensorFlow 2 프로젝트 워크플로우를 SageMaker에서 실행하기

- **본 노트북은 SageMaker example 중 tf-2-workflow부분 한글 번역입니다.**
- 원본소스 : https://github.com/aws-samples/amazon-sagemaker-script-mode/tree/master/tf-2-workflow

### Data Preprocessing -> Code Prototyping -> Automatic Model Tuning -> Deployment

1. [시작하기](#Introduction)
2. [SageMaker Processing 을 이용한 데이터셋 변환](#SageMakerProcessing)
3. [로컬모드 (Local Mode) 학습](#LocalModeTraining)
4. [로컬모드 (Local Mode) 엔드포인트](#LocalModeEndpoint)
5. [SageMaker 호스팅 환경에서 학습](#SageMakerHostedTraining)
6. [자동 모델 튜닝 (하이퍼파라미터 튜닝)](#AutomaticModelTuning)
7. [SageMaker 호스팅 엔드포인트](#SageMakerHostedEndpoint)
8. [AWS Step Functions의 Data Science SDK를 이용한 워크플로우 자동화](#WorkflowAutomation)
    1. [SageMaker Role에 IAM policy 추가하기](#IAMPolicy)
    2. [Step Functions를 위한 실행 역할(execution role) 생성하기](#CreateExecutionRole)
    3. [학습 파이프라인(TrainingPipeline)셋업](#TrainingPipeline)
    4. [워크플로우 가시화](#VisualizingWorkflow)
    5. [파이프라인의 생성과 실행 ](#CreatingExecutingPipeline)
    6. [리소스 정리](#Cleanup)
9. [확장](#Extensions)

    
## 시작하기 <a class="anchor" id="Introduction">

TensorFlow 2를 사용하실 때, SageMaker가 아닌 환경에서 사용하던 학습(training) 스크립트를 SageMaker에서 제공하는 TensorFlow 2 컨테이너에서 실행하도록 할 수 있습니다. SageMaker에서는 이 기능을 스크립트 모드라고 부릅니다. 스크립트 모드를 사용하여 여러분은 Tensorflow 2 프로젝트의 워크플로우를 완성할 수 있습니다.   
본 노트북은 그 워크플로우의 샘플을 제공합니다. 워크플로우는 SageMaker Processing을 이용한 전치리, SageMaker  로컬모드를 이용한 학습과 추론, 그리고 SageMaker에서 관리되는 학습과 추론환경을 이용하여 운영환경에 적용할 수 있는 모델을 학습하고 배포하는 과정을 포함합니다. 또, 모델의 하이퍼파라미터를 튜닝하기 위해 SageMaker의 Automatic Model Tuning을 이용할 수 있습니다. 추가로 [AWS Step Functions Data Science SDK](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/readmelink.html)를 이용하여 노트북 외부 환경에서 메인 학습과 배포 단계를 자동화할 수 있습니다. 

본 노트북을 실행하는데 1시간 정도가 소요됩니다. 시간내에 실행하기 위해 유즈케이스는 잘 알려진 보스톤의 집값을 예측하는 regression 작업을 이용합니다. 이 데이터셋은 방 개수, 고속도로 접근성, Charles강과의 거리 등 13개의 특성(feature)를 가집니다.

처음으로, 필요한 패키지를 임포트하고, local에서 학습과 테스트를 하기 위해 디렉토리를 세팅하는 작업을 시작합니다. 다양한 작업을 위해 SageMaker Session도 함께 셋업합니다. 입출력 데이터 저장을 위한 S3 버킷을 정의합니다. 본 예제에서는 SageMaker가 자동으로 생성하는 디폴트 버킷을 사용할 것입니다. 디폴트 버킷의 이름은 여러분의 Account ID와 리전을 포함하고 있습니다.


In [ ]:
import os
import sagemaker
import tensorflow as tf

sess = sagemaker.Session()
bucket = sess.default_bucket() 

data_dir = os.path.join(os.getcwd(), 'data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'data/test')
os.makedirs(test_dir, exist_ok=True)

raw_dir = os.path.join(os.getcwd(), 'data/raw')
os.makedirs(raw_dir, exist_ok=True)

## SageMaker Processing을 이용한 데이터셋 변환 <a class="anchor" id="SageMakerProcessing">

다음으로 필요한 데이터셋을 import하고 SageMaker Processing을 이용하여 변환하겠습니다. SageMaker Processing을 이용하면 SageMaker 노트북 서버와 분리된 독립적 환경에서 테라바이트 수준의 데이터를 변환처리할 수 있습니다. 전형적인 SageMaker 워크플로우에서 주로 프로토타이핑 작업에 사용되는 노트북은 고사양 인스턴스가 아닌 환경에서 주로 실행하고, 프로세싱, 학습, 모델 호스팅과 같은 실제 작업은 보다 강력한 사양의 인스턴스에서 실행합니다. SageMaker Processing은 Scikit-learn이 기본 탑재되어 있으며 Bring Your Own Container 옵션으로 다양한 데이터 변형기술과 작업을 별도로 독립된 환경에서 지원할 수 있습니다.

먼저 보스턴 집값 데이터셋을 로드하고, 이 데이터셋을 그대로 S3로 업로드한 후 SageMaker Processing에서 변형하겠습니다. 그리고 학습과 테스트를 위해 레이블도 생성, 저장합니다.

In [ ]:
import numpy as np
from tensorflow.python.keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

np.save(os.path.join(raw_dir, 'x_train.npy'), x_train)
np.save(os.path.join(raw_dir, 'x_test.npy'), x_test)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)

s3_prefix = 'tf-2-workflow'
rawdata_s3_prefix = '{}/data/raw'.format(s3_prefix)
raw_s3 = sess.upload_data(path='./data/raw/', key_prefix=rawdata_s3_prefix)
print(raw_s3)

SageMaker Processing을 사용하기 위해, 간단히 Python 데이터 프로세싱 스크립트를 만들어보겠습니다. 본 사례에서 우리는 SageMaker 에서 제공하는 Sckit-learn 컨테이너를 사용하겠습니다. 제공되는 컨테이너는 데이터 프로세싱을 위한 많은 함수들을 제공합니다. 여러분이 이 코드를 만들고 실행할 때에 약간의 제약사항이 있습니다. 입력과 출력 데이터는 분리된 디렉토리에 저장되어야 합니다. 이로부터 SageMaker Processing은 자동으로 입력 데이터를 S3로부터 로드하고 작업이 완료되면 변형이 완료된 데이터를 다시 S3로 업로드합니다.


<br> 

### TO DO
<span style="color:red"><b>힌트를 참고하여 아래 셀에서 TO DO를 완성합니다.</b></span>

<details>
    <summary>힌트</summary>
    
```python
import glob
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

if __name__=='__main__':
    
    input_files = glob.glob('{}/*.npy'.format('/opt/ml/processing/input'))
    print('\nINPUT FILE LIST: \n{}\n'.format(input_files))
    scaler = StandardScaler()
    for file in input_files:
        raw = np.load(file)
        transformed = scaler.fit_transform(raw)
        if 'train' in file:
            output_path = os.path.join('/opt/ml/processing/train', 'x_train.npy')
            np.save(output_path, transformed)
            print('SAVED TRANSFORMED TRAINING DATA FILE\n')
        else:
            output_path = os.path.join('/opt/ml/processing/test', 'x_test.npy')
            np.save(output_path, transformed)
            print('SAVED TRANSFORMED TEST DATA FILE\n')
```
</details>
<br>


In [ ]:
%%writefile preprocessing.py

import glob
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

if __name__=='__main__':
    
    # ----------------------------------------------------------------------
    # TO DO : 입력파일을 로드 경로를 지정합니다..
    # input_files = glob.glob('{}/*.npy'.format('/opt/ml/processing/input'))
    
    input_files = <TO DO>
    
    # ----------------------------------------------------------------------
       
    print('\nINPUT FILE LIST: \n{}\n'.format(input_files))
    scaler = StandardScaler()
    for file in input_files:
        raw = np.load(file)
        transformed = scaler.fit_transform(raw)
        
        if 'train' in file:
    
            # ----------------------------------------------------------------------     
            # TO DO : 출력파일이 저장될 경로를 지정합니다.
            # output_path = os.path.join('/opt/ml/processing/train', 'x_train.npy')
            
            output_path = <TO DO>
            
            # ----------------------------------------------------------------------       
            
            np.save(output_path, transformed)
            print('SAVED TRANSFORMED TRAINING DATA FILE\n')
        else:
            
            # ----------------------------------------------------------------------       
            # TO DO : 출력파일이 저장될 경로를 지정합니다.
            # output_path = os.path.join('/opt/ml/processing/test', 'x_test.npy')
            
            output_path = <TO DO>
            
            # ----------------------------------------------------------------------
            
            np.save(output_path, transformed)
            print('SAVED TRANSFORMED TEST DATA FILE\n')

SageMaker Processing 작업을 시작하기 전에, `SKLearnProcessor` 오브젝트를 만듭니다. 오브젝트 생성시 여러분은 작업에서 사용할 인스턴스 타입과 몇개의 인스턴스를 사용할 지 정의할 수 있습니다. 보스턴 집값 데이터셋은 매우 작은 데이터이지만 SageMaker Processing의 클러스터링 작업을 어떻게 샐행하는지 보기 위해 두 개의 인스턴스를 사용하겠습니다.  

In [ ]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=get_execution_role(),
                                     instance_type='ml.m5.xlarge',
                                     instance_count=2)

이제 Processing 작업을 실행할 준비가 되었습니다. 데이터 파일을 인스턴스에 동일하게 분산하기 위해 우리는 `ProcessingInput` 오브젝트에 `SharedByS3Key` 분산타입을 지정하겠습니다. 이 경우 만약 여러분이 `n`개의 인스턴스를 선언했다면, 각 인스턴스는 `1/n` 파일을 S3로부터 받게 될 것입니다. 다음 셀의 작업을 실행하는 데는 3분 정도 소요되며 대부분의 시간은 클러스터를 구성하는 시간입니다. 이 작업이 끝나면 클러스터는 SageMaker에 의해 자동으로 삭제될 것입니다. 

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

processing_job_name = "tf-2-workflow-{}".format(strftime("%d-%H-%M-%S", gmtime()))
output_destination = 's3://{}/{}/data'.format(bucket, s3_prefix)

sklearn_processor.run(code='preprocessing.py',
                      job_name=processing_job_name,
                      inputs=[ProcessingInput(
                        source=raw_s3,
                        destination='/opt/ml/processing/input',
                        s3_data_distribution_type='ShardedByS3Key')],
                      outputs=[ProcessingOutput(output_name='train',
                                                destination='{}/train'.format(output_destination),
                                                source='/opt/ml/processing/train'),
                               ProcessingOutput(output_name='test',
                                                destination='{}/test'.format(output_destination),
                                                source='/opt/ml/processing/test')])

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

위 SageMaker Processing 작업의 출력 로그에서 여러분은 두 가지 다른 색상으로 두 개의 서로다른 인스턴스의 로그를 볼 수 있고 각 인스턴스는 서로 다른 파일을 받은 것을 확인할 수 있습니다. 만약 `ShardedByS3Key` 분산타입을 사용하지 않았다면 각 인스턴스는 **모든** 파일의 복사본을 받았을 것입니다. 대부분의 stateless 데이터변형에서는 `n`개의 인스턴스에 동일하게 파일을 분산함으로써 댜략 `n`만큼 작업속도를 더 빠르게 할 수 있습니다.  

본 작업결과를 로컬에 저장한 후, 다음으로 로컬모드로 프로토타입 트레이닝과 추론코드를 작성하는 단계로 넘어가겠습니다.

In [ ]:
train_in_s3 = '{}/train/x_train.npy'.format(output_destination)
test_in_s3 = '{}/test/x_test.npy'.format(output_destination)
!aws s3 cp {train_in_s3} ./data/train/x_train.npy
!aws s3 cp {test_in_s3} ./data/test/x_test.npy

##  로컬모드 (Local Mode) 학습 <a class="anchor" id="LocalModeTraining">

SageMaker에서 로컬 모드는, 여러분이 작성한 코드를 SageMaker에서 관리되는 보다 강력한 클러스터에서 실행하기 전에, 여러분의 코드가 기대한 방식으로 동작하는 지 로컬에서 확인할 수 있는 편리한 방식입니다. 로컬모드 학습을 위해서는 docker-compose 또는 nvidia-docker-compose (GPU 인스턴스인 경우)의 설치가 필요합니다. 다음 셀의 명령은 본 노트북환경에 docker-compose 또는 nvidia-docker-compose를 설치하고 구성합니다. 

In [ ]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

다음으로 우리는 로컬모드 학습을 위해 Tensorflow Estimator를 구성할 것입니다. Estimator를 구성하는 주요 파라미터는 다음과 같습닏다.

- `train_instance_type`: 학습이 실행될 하드웨어 종류. CPU환경의서의 로컬모드는 단순히 `local` 로 지정합니다. GPU가 필요한 경우에는 `local_gpu`로 지정합니다. 
- `git_config`:  학습 스크립트가 팀에의해 협업, 공유되어 관리되는 경우 Estimator는 로컬 디렉토리가 아닌 Git 리포지토리에서 코드를 가져올 수 있습니다.
- Other parameters of note: Dictionary 형식으로 알고리즘의 하이퍼파리미터를 지정하고 Boolean 형식으로 Script Mode를 이용할 것인지를 지정합니다. 

여기서 우리는 주로 코드가 잘 동작하는지 확인하기 위해 로컬모드를 사용한다는 점을 기억하십시오. 우리는 불필요한 학습시간을 절약하기 위해, 전체 학습을 위한 큰 epoch 대신, 코드가 적절히 동작하는지 확인할 수 있는 정도의 작은 epoch으로 학습을 실행할 것입니다.  



<br> 

### TO DO
<span style="color:red"><b>train_model 디렉토리에 있는 train.py 파일을 열고 아래 힌트를 참고하여 TO DO를 완성합니다.</b></span>

<details>
    <summary>힌트</summary>
    
```python
import argparse
import numpy as np
import os
import tensorflow as tf

from model_def import get_model

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


def parse_args():
    
    parser = argparse.ArgumentParser()

    # 사용자가 전달한 하이퍼 파라미터를 command-line argument로 전달받아 사용함
    parser.add_argument('--epochs', type=int, default=1)
    parser.add_argument('--batch_size', type=int, default=64)
    parser.add_argument('--learning_rate', type=float, default=0.1)
    
    # data directories
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    
    # model directory: we will use the default set by SageMaker, /opt/ml/model
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    
    return parser.parse_known_args()


def get_train_data(train_dir):
    
    x_train = np.load(os.path.join(train_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(train_dir, 'y_train.npy'))
    print('x train', x_train.shape,'y train', y_train.shape)

    return x_train, y_train


def get_test_data(test_dir):
    
    x_test = np.load(os.path.join(test_dir, 'x_test.npy'))
    y_test = np.load(os.path.join(test_dir, 'y_test.npy'))
    print('x test', x_test.shape,'y test', y_test.shape)

    return x_test, y_test
   

if __name__ == "__main__":
    
    # 환경변수 또는 사용자 지정 hyperparameter로 전달된 argument를 읽는다.
    args, _ = parse_args()
    
    # training data를 가져온다. 위 코드에서 읽은 argument 중 'train'으로 전달된 값을 사용함. 
    # parse_args()를 통해 환경변수 'SM_CHANNEL_TRAIN'로 전달된 경로 'opt/ml/input/train/'가 'arg.train'으로 지정됨
    x_train, y_train = get_train_data(args.train)
    x_test, y_test = get_test_data(args.test)
    
    device = '/cpu:0' 
    print(device)
    batch_size = args.batch_size
    epochs = args.epochs
    learning_rate = args.learning_rate
    print('batch_size = {}, epochs = {}, learning rate = {}'.format(batch_size, epochs, learning_rate))

    with tf.device(device):
        
        model = get_model()
        optimizer = tf.keras.optimizers.SGD(learning_rate)
        model.compile(optimizer=optimizer, loss='mse')    
        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                  validation_data=(x_test, y_test))

        # evaluate on test set
        scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
        print("\nTest MSE :", scores)
                
        # 결과모델 저장 - 'args.model_dir'에는 'SM_MODEL_DIR' 환경변수를 통해 지정된 '/opt/ml/model/' 경로가 지정된다.
        model.save(args.model_dir + '/1')
    
```
</details>
<br>


In [ ]:
!pygmentize train_model/train.py

In [ ]:
from sagemaker.tensorflow import TensorFlow

git_config = {'repo': 'https://github.com/aws-samples/amazon-sagemaker-script-mode', 
              'branch': 'master'}

model_dir = '/opt/ml/model'
train_instance_type = 'local'
hyperparameters = {'epochs': 5, 'batch_size': 128, 'learning_rate': 0.01}
local_estimator = TensorFlow(git_config=git_config,
                             source_dir='tf-2-workflow/train_model',
                             entry_point='train.py',
                             model_dir=model_dir,
                             train_instance_type=train_instance_type,
                             train_instance_count=1,
                             hyperparameters=hyperparameters,
                             role=sagemaker.get_execution_role(),
                             base_job_name='tf-2-workflow',
                             framework_version='2.1',
                             py_version='py3',
                             script_mode=True)

아래의 `fit` 함수는 로컬 모드에서 학습작업을 시작할 것입니다. 학습을 위한 지표(Metric)은 코드의 아래에 로그로 보일 것입니다. 다섯 번의 epoch이 실행되는 동안 오류가 발생하지 않고 검증 오류(validation loss)가 점점 줄어드는 것을 통해 여러분의 학습코드가 기대한 것과 같이 동작한다는 것을 확인할 수 있습니다.

In [ ]:
inputs = {'train': f'file://{train_dir}',
          'test': f'file://{test_dir}'}

local_estimator.fit(inputs)

## entry_point script의 작성 (역주)

Tensorflow Estimator를 선언할 때 entry_point 부분을 주목해 주십시오. Tensorflow Estimator는 Tensorflow가 설치된 학습 실행환경은 aws에서 제공하는 도커 컨테이너 환경을 이용하면서, 딥러닝에 대한 부분은 사용자가 직접 코딩하고자 할 때 사용하는 방식입니다. 이 때 사용자기 직접 작성하신 될 train.py script가 에러없이 잘 동작하는지에 대한 확인이 필요해 지며, 이 때 로컬 모드를 통해 로컬(노트북) 리소스에서 코드를 실행하여 보다 빠르게 결과를 확인하고 디버깅하게 됩니다. 

그럼 해당 entry_point 스크립트는 어떻게 작성해야 할까요? 

### argument passing 로그 확인
Tensorflow Estimator가 동작하는 방식을 이해하기 위해, 방금 실행한 작업 로그에서 아래 코드를 찾아보십시오.

```bash
# Tensorflow 도커 컨테이너에서 entry_point 에 지정한 train.py 스크립트를 실행하는 부분 확인
/usr/bin/python3 train.py --batch_size 128 --epochs 5 --learning_rate 0.01 --model_dir /opt/ml/model
```

앞서 Tenorflow Estimator를 생성할 때 지정한 Hyperparameter와 model_dir이 train.py를 실행할 때 command-line argument로 전달된 것을 확인할 수 있습니다. 

또한 로그에서 `SM_INPUT_DIR`, `SM_CHANNEL_TRAIN` 등 `SM_...`로 지정된 환경변수로 어떤 값들이 세팅되었는지 확인합니다. 
```bash
algo-1-49mn2_1  | SM_HPS={"batch_size":128,"epochs":5,"learning_rate":0.01,"model_dir":"/opt/ml/model"}
algo-1-49mn2_1  | SM_USER_ENTRY_POINT=train.py
algo-1-49mn2_1  | SM_INPUT_DATA_CONFIG={"test":{"TrainingInputMode":"File"},"train":{"TrainingInputMode":"File"}}
algo-1-49mn2_1  | SM_OUTPUT_DATA_DIR=/opt/ml/output/data
algo-1-49mn2_1  | SM_CHANNELS=["test","train"]
algo-1-49mn2_1  | SM_INPUT_DIR=/opt/ml/input
algo-1-49mn2_1  | SM_OUTPUT_DIR=/opt/ml/output
algo-1-49mn2_1  | SM_MODEL_DIR=/opt/ml/model
algo-1-49mn2_1  | SM_CHANNEL_TRAIN=/opt/ml/input/data/train
algo-1-49mn2_1  | SM_CHANNEL_TEST=/opt/ml/input/data/test
algo-1-49mn2_1  | SM_HP_EPOCHS=5
algo-1-49mn2_1  | SM_HP_BATCH_SIZE=128
algo-1-49mn2_1  | SM_HP_LEARNING_RATE=0.01
algo-1-49mn2_1  | SM_HP_MODEL_DIR=/opt/ml/model
...
```


### entry point script 작성시 확인사항

entry_point를 통해 실행할 Tenosrflow script를 작성할 때에는 세 가지 사항을 이해해야 합니다. 코드와 함께 아래 내용을 확인합니다.

1. **입력(학습) 데이터 경로** - S3에 업로드된 학습데이터는 도커 컨테이너 내부의 `/opt/ml/input/data/{채널명}` 경로에 복제되므로 해당 위치에서 학습데이터를 읽어야 합니다. 여기서 채널명은 이후 셀에서 학습 실행(`fit`) 단계에 패스하게 되는 'train', 'test', 'validation' 등과 같은 구분값입니다. 해당 값은 도커 실행시 `SM_CHANNEL_TRAIN` 이라는 환경변수에 저장되어 있습니다. 본 예제에서는 해당 환경변수를 `args.train` argument를 통해 할당한 후 `get_train_data()` 함수 호출시 파라미터로 전달하는 구조를 사용하고 있습니다. (main함수 시작 부분의 코드를 참고하세요)

2. **출력(모델) 데이터 경로** - 학습이 완료되면 학습 리소스는 반납되므로, 학습결과로 생성된 모델은 다른 경로에 저장해 두어야 합니다. SageMaker에서는 도커 컨테이너 내부에서 `/opt/ml/model/`경로에 저장된 모델 파일을 S3로 export합니다. 또 학습 산출물을 export하기 위한 `/opt/ml/output/`경로의 파일들도 S3로 export 됩니다. (NLP 문제일 경우 voacabulary, 생성이미지, 그래프, 별도 로그 등 작업에 따라 다양한 결과물을 처리할 수 있음.) 본 예제에서는 `SM_MODEL_DIR` 환경변수를 이용하여 main함수의 맨아래 줄에서 해당 경로에 모델을 저장하고 있습니다.

3. **하이퍼파라미터 argument** - 학습실행을 요청하는 노트북환경에서 하이퍼파리미터 등을 제어할 경우에도 마찬가지 방식으로 command-line argument를 이용합니다. Estimator 선언시 Hyperparamenter로 지정한 값들은 entry point 스크립트 실행시 사용자 정의 argument의 형태로 전달됩니다. 위 코드에서 (앞 단계 Estimator 지정시 선언한 하이퍼파라미터인) `epoch`, `batch_size`, `learning_rate`을 parsing하는 부분을 확인합니다.

entry_point 스크립트 작성에 대한 보다 자세한 내용은 다음 SDK 문서를 참고하십시오.  
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#train-a-model-with-tensorflow 



## 로컬모드 (Local Mode) 엔드포인트 <a class="anchor" id="LocalModeEndpoint">

SageMaker에서 로컬모드 학습은 전체 데이터를 이용하여 학습을 하기 전에 학습코드를 확인하는 용도로 매우 유용한 것과 같이, 여러분의 모델을 고사양의 자원이 있는 실제 운영환경에 배포하여 추론을 실행하기 전에 로컬 환경에서 검증하는 것 또한 유용할 수 있습니다. 이를 위해 Tensorflow SaveModel 아티펙트(artifact) 또는 S3에 저장된 모델 체크포인트를 가져와서(fetch) 노트북환경에 로드하고 검증하는 방식을 사용할 수 있습니다. 하지만 이보다 편리한 방법은 SageMaker Python SDK를 이용하여 로컬모드 엔드포인트(Local Mode endpoint)를 구성하는 것입니다. 
    
보다 구체적으로, 로컬모드 학습을 이용하여 만들어진 Estimator 오브젝트는 로컬환경에 모델을 배포(deploy)할 수 있습니다. 이를 위해 로컬모드 학습에서 처럼 여러분의 노트북이 GPU 인스턴스인지 CPU인스턴스인지에 따라 인스턴스 타입을 `local_gpu` 또는 `local` 로 지정하고 로컬 Estimator의 배포모델을 invoke합니다.
    
다음 한 줄의 코드로 SageMaker Tensorflow Serving 컨테이너를 로컬에 deploy할 수 있습니다.
    

In [ ]:
local_predictor = local_estimator.deploy(initial_instance_count=1, instance_type='local')

<br>
로컬모드 엔드포인트에서 예측을 실행하기 위해, 앞 단계에서 생성된 Predictor의 predict 함수를 호출합니다.

In [ ]:
local_results = local_predictor.predict(x_test[:10])['predictions']

예측결과(predictions)를 실제 값(target values)과 비교합니다. 

In [ ]:
local_preds_flat_list = [float('%.1f'%(item)) for sublist in local_results for item in sublist]
print('predictions: \t{}'.format(np.array(local_preds_flat_list)))
print('target values: \t{}'.format(y_test[:10].round(decimals=1)))

현재까지는 적은 epoch으로 학습을 실행했기 때문에 개선이 필요합니다. 하지만 예측값은 어느정도 합리적인 범위에서 리턴되어야 합니다.

SageMaker Tensorflow Serving 컨테이너가 로컬에서 무한정 실행되는 것을 막기 위해, delete_endpoint()를 호출하여 중지시킵니다. 

In [ ]:
local_predictor.delete_endpoint()

##  SageMaker 호스팅 환경에서 학습 <a class="anchor" id="SageMakerHostedTraining">

지금까지 로컬에서 코드가 실행되는 것을 확인했습니다. 이제 SageMaker가 호스팅하는 학습기능을 사용해보겠습니다. SageMaker가 호스팅하는 학습환경은 데이터의 사이즈가 크고, 분산학습이 필요한 등 보다 실제적인 학습에서 선호하는 환경일 것입니다. SageMaker가 호스팅하는 환경에서는 로컬 모드에서 처럼 노트북 인스턴스에서 학습이 실행되는 것이 아니라 SageMaker가 관리하는 별도의 클러스터에서 실행됩니다. 학습을 시작하기 전에 데이터는 S3 또는 EFS, FSx for Lustre 파일시스템에 저장되어 있어야 합니다. 본 예제에서는 S3를 사용할 것이므로 S3에 데이터를 업로드하고 작업이 잘 되었는지 확인합니다.

In [ ]:
s3_prefix = 'tf-2-workflow'

traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)

In [ ]:
train_s3 = sess.upload_data(path='./data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./data/test/', key_prefix=testdata_s3_prefix)

inputs = {'train':train_s3, 'test': test_s3}

print(inputs)

이제 SageMaker 호스팅 환경에서 학습을 위해 Estimator를 셋업합니다. 이것은 로컬모드의 Estimator와 유사합니다만, 한가지 차이는 `train_instance_type` 항목이 `local`이 아닌 SageMaker ML 인스턴스 타입이 들어간다는 점입니다. 그리고, 우리는 우리의 코드가 잘 작동하는 것을 검증했으므로, 이번에는 보다 개선된 성능(보다 적은 validation loss)을 얻을 수 있도록 많은 epoch으로 작업을 실행하겠습니다. 

이 두 가지를 변경한 후 단순히 `fit`명령을 호출합니다.

In [ ]:
train_instance_type = 'ml.c5.xlarge'
hyperparameters = {'epochs': 30, 'batch_size': 128, 'learning_rate': 0.01}

estimator = TensorFlow(git_config=git_config,
                       source_dir='tf-2-workflow/train_model',
                       entry_point='train.py',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-2-workflow',
                       framework_version='2.1',
                       py_version='py3',
                       script_mode=True)

아래의 `fit`명령을 실행하여 SageMaker 호스팅환경에서의 학습을 시작한 후, epoch이 진행되면서 학습작업이 수렴하는지 관찰합니다. 로컬모드에서의 학습에 비해 검증 오류(validation loss)가 현저히 줄어드는지 확인합니다. 이보다 개선된 성능튜닝은 이후 **Automatic Model Tuning** 파트에서 다시 다루게 됩니다.

In [ ]:
estimator.fit(inputs)

로컬모드에서의 학습처럼 호스팅 환경에서의 학습 또한 결과 모델을 생성하고 S3에 저장하여 사용자가 쉽게 꺼내올 수 있습니다. SageMaker의 기능들은 모듈화 되어 있어서 SageMaker에서 학습한 모델을 가져와서 다른 환경에서 실행할 수도 있고 SageMaker 호스팅 환경에서 실행하 수도 있습니다. SageMaker 호스팅 환경에서 실행하는 방법은 본 노트북의 **SageMaker hosted endpoint** 파트에서 보실수 있습니다.

S3로부터 모델을 가져올 때에는, 학습에 사용한 estimator에서 결과모델의 경로를 저장하고 있기 때문에 이를 참조하면 됩니다. estimator의 `model_data`속성에 저장된 S3 경로로부터 모델을 가져와서 압축을 해제하는 간단한 절차로 바로 내용 확인이 가능합니다. 


In [ ]:
!aws s3 cp {estimator.model_data} ./model/model.tar.gz

아카이빙된 모델 파일에는 .pb파일 등 Tensorflow Serving에서 추론을 실행하기 위한 필요한 리소스들이 포함되어 있습니다. 

In [ ]:
!tar -xvzf ./model/model.tar.gz -C ./model

## 자동 모델 튜닝 (하이퍼파라미터 튜닝) <a class="anchor" id="AutomaticModelTuning">
    
지금까지 로컬모드에서 학습과 호스팅환경에서의 학습을 진행했습니다. 이 과정에서 epoch을 늘리는 것 이외에는 아직 더 나은 모델을 위한 하이퍼파라미터 튜닝은 진행하지 않았습니다. 적절한 하이퍼파라미터 값을 선택하는 것을 수작업으로 진행할 때에는 일반적으로 많은 시간이 필요하고 쉽지 않은 작업이 됩니다. 적절한 하이퍼파라미터의 조합은 데이터와 알고리즘에 따라 다릅니다. 어떤 알고리즘은 매우 다양한 하이퍼파라미터를 가지기도 하고, 어떤 알고리즘은 선택되는 하이퍼파라미티에 의해 매우 민감하게 동작하기도 합니다. 그리고 대부분의 경우 모델의 학습과 하이퍼파리미터 값의 관계는 비선형적입니다. SageMaker 자동 모델 튜닝(Automatic Model Tuning)은 이런 하이퍼파라미터 튜닝을 자동화하는 것을 도와줍니다. 자동 모델 튜닝 기능을 통해 하이퍼파라미터의 조합에 따라 여러개의 학습작업을 병렬로 실행하고 최고의 성능을 내는 하이퍼파라미터의 조합을 찾을 수 있습니다. 

그럼 튜닝하고자하는 하이퍼파라미터를 선택하고 가각 값의 범위를 지정하는 것부터 시작하겠습니다. 그다음 최적화를 위한 목표 지표(metric)을 지정합니다. 본 예제에서는 검증오류(validation loss)를 최소화하도록 설정하겠습니다. 

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
  'learning_rate': ContinuousParameter(0.001, 0.2, scaling_type="Logarithmic"),
  'epochs': IntegerParameter(10, 50),
  'batch_size': IntegerParameter(64, 256),
}

metric_definitions = [{'Name': 'loss',
                       'Regex': ' loss: ([0-9\\.]+)'},
                     {'Name': 'val_loss',
                       'Regex': ' val_loss: ([0-9\\.]+)'}]

objective_metric_name = 'val_loss'
objective_type = 'Minimize'

다음은 앞서 지정한 파라미터를 처리할 HyperparameterTuner 오브젝트를 선언합니다. 튜닝작업 설정시 최대로 실행할 학습작업의 수를 지정하게 됩니다. 하이퍼파라미터 튜닝 작업은 해당 숫자만큼 실행된 후 완료될 것입니다. 

동시에 병렬처리할 작업수도 함께 지정합니다. 본 예제에서는 5개로 설정하였습니다. 이렇게 세팅(max_job=15, max_parallel_jobs=5)할 경우, 하이퍼파라미터 튜닝 작업은 5개의 병렬작업을 가지는 3개의 시리즈로 튜닝작업을 실행하게 됩니다. 튜닝 전략은 디폴트로 베이지안 최적화(Bayesian Optimization)를 사용하며, 다음 학습작업은 이전 학습작업 그룹의 결과에 따라 적절한 하이퍼파리미터값을 찾게 됩니다. 때문에 모든 작업을 병렬로 실행하는 것보다 적절한 그룹으로 나누는 것이 좋습니다. 병렬작업수의 지정은 장단점이 있습니다. 보다 많은 병렬작업을 지정하면 전체 작업은 빨리 종료될 것이지만 이전 튜닝작업에서 찾은 정확도를 희생하게 됩니다.


이제 HyperparameterTuner 오브젝트의 `fit`명령을 통해 하이퍼파라미터 튜닝작업을 실행합니다. 튜닝작업은 10분 정도 걸립니다. 작업결과를 기다리는 동안 SageMaker 콘솔의 **Hyperparameter tuning jobs** 항목을 통해 개별 학습작업의 메타데이터와 그 실행결과 등 튜닝 작업의 상태를 확인할 수 있습니다.


In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=15,
                            max_parallel_jobs=5,
                            objective_type=objective_type)

tuning_job_name = "tf-2-workflow-{}".format(strftime("%d-%H-%M-%S", gmtime()))
tuner.fit(inputs, job_name=tuning_job_name)
tuner.wait()

튜닝작업이 완료된 후에는 SageMaker Python SDK의 `HyperparameterTuningJobAnalytics` 오브젝트를 통해 성능 순서로 튜닝작업을 리스트업해 봅니다. 다양한 튜닝작업 결과 중 최고의 검증 오류(validation loss)를 가지는 top 5 작업을 조회합니다. 그리고 FinalObjectiveValue 컬럼을 참고하십시오. 튜닝작업마다 작업결과는 다양하지만 최고로 선정된 튜닝작업의 검증오류는 호스팅 환경에서 실행했던 (단순히 epoch값만 조정했던) 학습작업에 비해 현저히 낮은 것을 확인할 수 있습니다.


In [ ]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=True).head(5)

전체 학습시간과 학습작업 상태는 다음 라인의 코드로 확인할 수 있습니다. 자동 조기종료(early stopping) 여부의 디폴트 설정이 off 이기 때문에, 학습작업은 모두 완전하게 실행되었을 것입니다. 자세한 튜닝작업의 예제는 다음 [HPO_Analyze_TuningJob_Results.ipynb](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb) 노트북을 참조합니다. 

In [ ]:
total_time = tuner_metrics.dataframe()['TrainingElapsedTimeSeconds'].sum() / 3600
print("The total training time is {:.2f} hours".format(total_time))
tuner_metrics.dataframe()['TrainingJobStatus'].value_counts()

## SageMaker 호스팅 엔드포인트 <a class="anchor" id="SageMakerHostedEndpoint">

자동 모델 튜닝작업에서 학습된 결과는 간단한 과정으로 운영환경에 배포할 수 있습니다. 가장 간편한 방식은 SageMaker 호스팅 엔드포인트를 사용하여 학습모델로부터 실시간 예측서비스를 구현하는 것입니다. (배치 변환 작업(Batch Transform jobs) 또한 비동기방식으로 대량 데이터에 대해 오프라인 예측을 실행할 수 있는 좋은 방식입니다.) 엔드포인트는 학습작업에서 생성한 Tensorflow 저장모델을 가져와서 Tensorflow 서빙 컨테이너에 배포할 것입니다. 이 작업은 다음 한 줄의 코드로 가능합니다.
     
보다 구체적으로, HyperparameterTuner 오브젝트의 `deploy` 명령을 호출하면 해당 튜닝 작업에서 최고로 선택된 모델을 SageMaker 호스팅 엔드포인트에 배포하게 됩니다. 호스팅환경으로의 배포 작업은 로컬모드 엔드포인트의 작업에 비하여 수분이상 더 소요될 수 있습니다. (추론코드 프로토타이핑이 목적일 때에는 로컬모드도 좋은 옵션일 수 있습니다.)
     

In [ ]:
tuning_predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

방금 배포한 엔드포인트에서 예측을 실행해 봅니다. 

In [ ]:
results = tuning_predictor.predict(x_test[:10])['predictions'] 
flat_list = [float('%.1f'%(item)) for sublist in results for item in sublist]
print('predictions: \t{}'.format(np.array(flat_list)))
print('target values: \t{}'.format(y_test[:10].round(decimals=1)))

추가 요금이 청구되지 않도록 엔드포인트를 삭제합니다.

In [ ]:
sess.delete_endpoint(tuning_predictor.endpoint)

## AWS Step Functions의 Data Science SDK를 이용한 워크플로우 자동화 <a class="anchor" id="WorkflowAutomation">

지금까지 Tensorflow 프로젝트의 다양한 단계들을 프로토타입의 형태로 살펴보았습니다. 노트북은 프로토타입 작업에는 매우 훌륭한 도구이지만 일반적으로 운영환경의 머신러닝 파이프라인에서는 잘 사용되지 않습니다. 예를 들어 SageMaker에서 간단한 파이프라인을 만든다면 다음 단계들을 포함할 것입니다:

1. 모델을 학습한다. 
2. 서빙을 위한 모델 아티팩트를 담고 있는 SageMaker 모델 오브젝트를 생성한다. 
3. 모델이 서빙될 때 필요한 설정(하드웨어 타입과 수량 등)을 담고 있는 SageMaker 엔드포인트 구성(Endpoint Configuration)을 생성한다.
4. 해당 구성을 이용하여 학습된 모델을 SageMaker Endpoint에 배포한다. 
    
AWS Step Functions 에서 제공하는 Data Science SDK는 이런 워크플로우를 생성하는 것을 자동화해 줍니다. 간단한 파이썬 스크립트 작성으로 워크플로우의 각 단계와 이들의 연결을 정의하여 구현하는 방식입니다. 모든 워크플로우 단계는 Step Functions에서 처리되며, 관리형 서비스이므로 인프라에 대한 설치나 운영은 고민하지 않아도 됩니다. 

워크플로우 생성을 위해 Step Functions Data Science SDK를 설치합니다.

In [ ]:
import sys

!{sys.executable} -m pip install --quiet --upgrade stepfunctions

### SageMaker Role에 IAM policy 추가하기 <a class="anchor" id="IAMPolicy">

**SagrMaker 노트북 인스턴스에서 본 단계를 실행할 때에는** 해당 노트북 인스턴스에서 Step Functions의 워크플로우를 생성하고 실행하는 권한부여 위해 IAM 역할(Role) 설정이 필요합니다.
    
1. [SageMaker console](https://console.aws.amazon.com/sagemaker/)을 오픈합니다.
2. **노트북 인스턴스(Notebook instances)**를 선택하고 사용중인 노트북 인스턴스의 이름을 클릭합니다.
3. **권한 및 암호화** 에서 IAM 역할 ARN을 선택하여 IAM 콘솔의 역할 화면으로 이동합니다.
4. **정책 연결** 을 클릭하고 `AWSStepFunctionsFullAccess` 정책을 검색합니다.
5. `AWSStepFunctionsFullAccess`옆의 체크박스틀 체크하고 **Attach policy**를 클릭합니다.
    
현재 실행중인 노트북이 (SageMaker 인스턴스가 아닌) 다른 환경이라면, SDK는 AWS CLI 구성을 사용할 것입니다. 보다 자세한 내용은 [Configuring the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html)를 참고합니다.

### Step Functions를 위한 실행 역할(execution role) 생성하기 <a class="anchor" id="CreateExecutionRole">

다음단계로 Step Functions에서 SageMaker와 다른 서비스 기능에 접근할 수 있도록 실행 역할(execution role)을 생성합니다.
    
1. [IAM 콘솔](https://console.aws.amazon.com/iam/)로 이동합니다.
2. **역할(Roles)**와 **역할 생성(Create role)**을 클릭합니다.
3. **또는 서비스를 선택하여 해당 서비스의 사용 사례 확인(Choose the service that will use this role)**에서 **Step Functions**를 선택합니다.
4. **역할 이름(Role name)**을 선택하는 화면이 나올 때까지 **다음(Next)**을 클릭합니다.
5. 역할이름을 `StepFunctionsWorkflowExecutionRole`와 같이 입력하고 **역할 생성(Create role)**을 클릭합니다.
    
새롭게 생성된 역할을 선택하고 정책을 연결합니다. 다음 단계는 Step Functions에 접근하기 위한 전체 권한을 부여하지만 실제 업무환경에서 베스트 프랙티스는 필요한 리소스에만 권한을 연결하는 것입니다. 

Select your newly create role and attach a policy to it. The following steps attach a policy that provides full access to Step Functions, however as a good practice you should only provide access to the resources you need.  

1. **권한(Permissions)** 탭에서 **인라인 정책 추가(Add inline policy)**를 클릭합니다.
2. **JSON** 탭에 다음 코드를 붙여넣습니다.


```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateTransformJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:StopTransformJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DeleteEndpoint",
                "sagemaker:UpdateEndpoint",
                "sagemaker:ListTags",
                "lambda:InvokeFunction",
                "sqs:SendMessage",
                "sns:Publish",
                "ecs:RunTask",
                "ecs:StopTask",
                "ecs:DescribeTasks",
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "batch:SubmitJob",
                "batch:DescribeJobs",
                "batch:TerminateJob",
                "glue:StartJobRun",
                "glue:GetJobRun",
                "glue:GetJobRuns",
                "glue:BatchStopJobRun"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        }
    ]
}
```

3. **정책 검토(Review policy)**를 선택하고 정책이름을 `StepFunctionsWorkflowExecutionPolicy`등으로 부여합니다.
4. **정책 생성(Create policy)**을 클릭하고 역할 상세 페이지로 이동합니다.
5. **요약(Summary)** 아래의 내용에서 **역할(Role) ARN**을 복사합니다.
    

### 학습 파이프라인(TrainingPipeline)셋업 <a class="anchor" id="TrainingPipeline">

AWS Step Functions의 Data Science SDK 이용시 다양한 명령을 제공하여 파이프라인을 처음부터 만들수도 있고, 또는 보편적인 워크플로우를 위해 미리 제공되는 템플릿 방식을 이용할 수도 있습니다. 예를 들어 [TrainingPipeline](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/pipelines.html#stepfunctions.template.pipeline.train.TrainingPipeline) 오브젝트는 학습과 배포를 포함하는 기본 파이프라인의 생성을 도와줍니다.
    
다음 셀의 코드는 이 기본 파이프라인을 위한 `pipeline`오브젝트를 생성하며 이를 위한 필수 파라미터를 설정하고 있습니다.


In [ ]:
import stepfunctions

from stepfunctions.template.pipeline import TrainingPipeline

# paste the StepFunctionsWorkflowExecutionRole ARN from above
workflow_execution_role = "<execution-role-arn>"

pipeline = TrainingPipeline(
    estimator=estimator,
    role=workflow_execution_role,
    inputs=inputs,
    s3_bucket=bucket
)

### 워크플로우 가시화 <a class="anchor" id="VisualizingWorkflow">

이제 워크플로우 정의를 그래프의 형태로 가시화하여 확인할 수 있습니다. 이 워크플로우는 학습의 시작부터 배포까지의 과정을 파이프라인으로 연결하고 있습니다.


In [ ]:
print(pipeline.workflow.definition.to_json(pretty=True))

In [ ]:
pipeline.render_graph()

### 파이프라인의 생성과 실행 <a class="anchor" id="CreatingExecutingPipeline">

워크플로우를 실행하기 위해 `create`명령을 이용하여 파이프라인을 클라우드 환경에 생성합니다.


In [ ]:
pipeline.create()

워크프로우를 시작하기 위해 `execute`명령을 실행합니다.

In [ ]:
execution = pipeline.execute()

`list_executions`명령을 사용하여 워크플로우의 모든 실행을 리스트업할 수 있습니다. 생성된 파이프라인은 새로운 데이터에 대한 재학습 등 필요한 만큼 반복적으로 실행 가능합니다. (본 노트북은 한번만 실행합니다.) 작업의 결과는 AWS Step Functions 콘솔에서 접근할 수 있습니다.


In [ ]:
pipeline.workflow.list_executions(html=True)

워크플로우가 실행되는 동안 `render_progress`명령을 통해 진행상태를 체크할 수 있습니다. 이 명령은 워크플로우 현재 상태에 대한 정적 이미지 스냅샷을 생성합니다. 워크플로우가 실행되는 동안 다음 셀을 반복적으로 실행해 봅니다. 


In [ ]:
execution.render_progress()

#### 노트북의 나머지 부분을 진행하기 전에:

워크플로우가 **Succeeded**상태로 종료될 때까지 기다립니다. 이 작업은 수분 정도가 걸릴 것입니다. 위 `render_progress`명령셀의 결과에서 **Inspect in AWS Step Functions**링크를 클릭하면 브라우저를 통해 상태를 확인할 수 있습니다. 

완료된 워크플로우의 상세정보를 보기 위해 `list_events`명령을 사용할 수 있습니다. 이 명령은 워크플로우 실행과 관련된 모든 이벤트를 리스트업합니다.


In [ ]:
execution.list_events(reverse_order=True, html=False)

From this list of events, we can extract the name of the endpoint that was set up by the workflow.  
결과로 나오느 이벤트 목록에서 워크플로우의 엔드포인트 이름을 찾을 수 있습니다.

In [ ]:
import re

endpoint_name_suffix = re.search('endpoint\Wtraining\Wpipeline\W([a-zA-Z0-9\W]+?)"', str(execution.list_events())).group(1)
print(endpoint_name_suffix)

엔드포인트 이름을 알게 되면, 이 enpoint와 연결된 TensorFlowPredictor 오브젝트를 생성할 수 있습니다. 이 오브젝트는 다음 코드셀의 예시처럼 예측을 제공합니다.

#### 다음 코드 셀을 실행하기 전에:

[SageMaker 콘솔](https://console.aws.amazon.com/sagemaker/)로 이동하여 **엔드포인트(Endpoints**를 선택하고 엔드포인트의 상태가 **InService** 상태인지 확인합니다. 만약 **Creating**상태라면 상태가 바뀔 때까지 기다립니다. 이 단계는 수 분이 걸릴 수 있습니다.


In [ ]:
from sagemaker.tensorflow import TensorFlowPredictor

workflow_predictor = TensorFlowPredictor('training-pipeline-' + endpoint_name_suffix)

results = workflow_predictor.predict(x_test[:10])['predictions'] 
flat_list = [float('%.1f'%(item)) for sublist in results for item in sublist]
print('predictions: \t{}'.format(np.array(flat_list)))
print('target values: \t{}'.format(y_test[:10].round(decimals=1)))

Using the AWS Step Functions Data Science SDK, there are many other workflows you can create to automate your machine learning tasks.  For example, you could create a workflow to automate model retraining on a periodic basis.  Such a workflow could include a test of model quality after training, with subsequent branches for failing (no model deployment) and passing the quality test (model is deployed).  Other possible workflow steps include Automatic Model Tuning, data preprocessing with AWS Glue, and more.  



For a detailed example of a retraining workflow, see the AWS ML Blog post [Automating model retraining and deployment using the AWS Step Functions Data Science SDK for Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/automating-model-retraining-and-deployment-using-the-aws-step-functions-data-science-sdk-for-amazon-sagemaker/).

### 리소스 정리 <a class="anchor" id="Cleanup">

방금 우리는 워크플로우를 통해 모델과 엔드파인트를 SageMaker에 배포하였습니다. 사용하지 않는 엔드포인트에 대한 추가 과금이 청구되지 않도록 엔드포인트를 삭제합니다. [SageMaker 콘솔](https://console.aws.amazon.com/sagemaker/) 로 이동하여 왼쪽 패털에서 **엔드포인트(Endpoints)**를 선택하고 리스트에서 사용하지 않는 엔드포인트를 선택하고 삭제합니다.

## 확장 <a class="anchor" id="Extensions">

본 노트북에서 다음을 다루었습니다.: 데이터 변환을 위한 SageMaker Processing, 학습과 추론에 대한 로컬모드 프로토타이핑, 자동 모델 튜닝(하이퍼파라미터 튜닝), SageMaker 호스팅 환경에서 학습과 추론. 이들은 대부분의 SageMaker에서 딥러닝 워크플로우에서 모두 핵심이 되는 요소들입니다. 추가로 우리는 프로젝트에서 프로토타이핑이 끝난 후, AWS Step Functions Data Science SDK를 이용하여 워크플로우 생성하고 실행하는 부분을 살펴보았습니다.
    
지금까지 살펴본 기능 외에도, SageMaker에는 여러분의 프로젝트에 적용 가능한 다른 많은 기능들이 있습니다. 예를 들어 딥러닝 모델의 학습시 발생하는 공통적인 문제들을 다루기 위해 제공되는 **SageMaker 디버거(Debugger)** 기능이 있으며 모델이 운영환경에 적용되었을 때 발생하는 공통적인 문제들을 관리하기 위한 **SageMaker Model Monitor** 와 같은 기능들이 유용할 수 있습니다.
